<a href="https://colab.research.google.com/github/Kira1108/langchain-experiments/blob/main/Lanchain_Chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython.display import clear_output
!pip install openai langchain
clear_output()

In [2]:
import os
import openai
os.environ['OPENAI_API_KEY'] = ""

In [3]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

### Compose Chains

In [16]:
llm = ChatOpenAI(temperature = 1)

prompt_product = PromptTemplate(template = "What is a fantastic and creative company name in Chinese that sells product {product}", input_variables = ['product'])
chain_product = LLMChain(prompt = prompt_product, llm = llm, verbose = True)


prompt_phrase = PromptTemplate(input_variables=["company_name"],template="Write a catchphrase in Chinese for the following company: {company_name}")
chain_phrase = LLMChain(llm=llm, prompt=prompt_phrase, verbose = True)

In [17]:
chain_phrase.run(chain_product.run("中国山水画"))



> Entering new LLMChain chain...
Prompt after formatting:
What is a fantastic and creative company name in Chinese that sells product 中国山水画

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Write a catchphrase in Chinese for the following company: 青山水画 (Qīngshān shuǐhuà) which means "Green Mountain Landscape Painting".

> Finished chain.


'"一幅青山画，一描山水情" (Yī fú qīngshān huà, yī miáo shānshuǐ qíng) - which translates to "A green mountain painting, captures the essence of nature".'

In [18]:
chain_phrase.run(chain_product.run("宠物店"))



> Entering new LLMChain chain...
Prompt after formatting:
What is a fantastic and creative company name in Chinese that sells product 宠物店

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Write a catchphrase in Chinese for the following company: 萌宠之家 (Měng chǒng zhī jiā), which translates to "Adorable Pet Home."

> Finished chain.


'"小宠物，大幸福" (xiǎo chǒngwù, dà xìngfú) which means "Small pets, big happiness."'

In [20]:
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain_product, chain_phrase], verbose=True)

catchphrase = overall_chain.run("中国臭豆腐")



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
What is a fantastic and creative company name in Chinese that sells product 中国臭豆腐

> Finished chain.
香炸阁 (xiāng zhà gé) - which translates to "Fragrant Fried Pavilion".


> Entering new LLMChain chain...
Prompt after formatting:
Write a catchphrase in Chinese for the following company: 香炸阁 (xiāng zhà gé) - which translates to "Fragrant Fried Pavilion".

> Finished chain.
享受香炸美食，尽在香炸阁！
(Enjoy the delicious fragrance of fried food, all at Fragrant Fried Pavilion!)

> Finished chain.


**Chain是顶层对象，你需要实现下面几个方法**

In [24]:
from langchain.chains import LLMChain
from langchain.chains.base import Chain

from typing import Dict, List

'''
# Chain is a BaseModel
from pydantic import BaseModel
>>> issubclass(Chain, BaseModel)
>>> True
'''

class ConcatenateChain(Chain):
    # Standard way of writing BaseModel subclasses
    chain_1: LLMChain
    chain_2: LLMChain

    @property
    def input_keys(self) -> List[str]:
        # Union of the input keys of the two chains.
        all_input_vars = set(self.chain_1.input_keys).union(set(self.chain_2.input_keys))
        return list(all_input_vars)

    @property
    def output_keys(self) -> List[str]:
        return ['concat_output']

    def _call(self, inputs: Dict[str, str]) -> Dict[str, str]:
        output_1 = self.chain_1.run(inputs)
        output_2 = self.chain_2.run(inputs)
        return {'concat_output': output_1 + output_2}

In [25]:
class ExtractEntityChain(Chain):
    entity_chain:LLMChain
    relation_chain:LLMChain

    @property
    def input_keys(self) -> List[str]:
        # Union of the input keys of the two chains.
        all_input_vars = set(self.chain_1.input_keys).union(set(self.chain_2.input_keys))
        return list(all_input_vars)

    @property
    def output_keys(self) -> List[str]:
        return ['final_result']

In [86]:
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain, LLMChain

llm = OpenAI(temperature = 0)

entities_extraction_prompt_template = """
Extract all entities names (only person, location, and organization) from the following text,
Entities are normally nouns, prouns are all ignored, for example, he, she, this, that.
Only determined entities which has a name is extracted.
Give the result as a list, separated by comma

EXAMPLE
text:  小明和小红今在重庆大学上学
entities: 小红, 小明, 重庆大学
EXAMPLE END

text: {text}
entities:
"""

entities_extract_prompt = PromptTemplate(template = entities_extraction_prompt_template, input_variables = ['text'])

extraction_chain = LLMChain(llm = llm, prompt = entities_extract_prompt)

entities = extraction_chain.run("哎卧槽，我老王觉得，这种时候，我们就别维护什么对话历史了吧，langchain真的垃圾")

entities = [e.strip() for e in entities.split(",")]

In [88]:
entities

['老王', 'langchain']